<a href="https://colab.research.google.com/github/jsstar522/text_mining/blob/master/01_Basic/Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crawling for Python

## Package 설치

### Beatiful Soup 4
로컬로 HTML문서를 가져올 수 있는 HTML parser입니다.

### LXML
로컬로 XML문서를 가져올 수 있는 XML parser입니다.

In [13]:
!pip install lxml

    100% |████████████████████████████████| 5.8MB 4.9MB/s 


### Requests

HTTP를 이용해서 웹과 통신하는 Package입니다.

## 네이버 영화 페이지에서 영화제목 가져오기

네이버 영화 페이지의 **보헤미안 렙소디** : https://movie.naver.com/movie/bi/mi/basic.nhn?code=156464

이렇게 다양한 list를 보여주는 대부분의 웹페이지는 페이지 url마다 숫자로 된 id를 가지고 있습니다. 하지만 그렇지 않은 웹페이지도 많기 때문에 **페이지를 잘 살펴보는 것이 크롤링의 첫번째 과제입니다**. 영화제목을 가져오기 위해서는 이렇게 영화목록을 클릭해서 제목을 가져와야합니다. 즉, 다양한 페이지의 url_id를 변수로 둬야합니다.



In [0]:
from bs4 import BeautifulSoup
import os
import re
import requests
import lxml
import sys
from pprint import pprint
## pprint = pretty print, 더 깔끔한 출력을 위한 module

page_url_base = ' https://movie.naver.com/movie/bi/mi/basic.nhn?code=%s'

movie_url_id = 156464

In [2]:
page_url = page_url_base % movie_url_id

## 해당 url 웹페이지의 정보를 가져온다
r = requests.get(page_url)

html = r.text

## requests로 가져온 페이지 정보를 lxml로 보기좋게 구조화
page = BeautifulSoup(html, 'lxml')

r.headers


## lxml 사용 오류가 날 때, 런타임 재시작할 것

{'Date': 'Mon, 03 Dec 2018 17:01:59 GMT', 'Pragma': 'no-cache', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Cache-Control': 'no-cache, no-store', 'Content-Language': 'ko-KR', 'P3P': 'CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC", CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC"', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '38192', 'Content-Type': 'text/html;charset=UTF-8', 'Referrer-Policy': 'unsafe-url', 'Server': 'nfront'}

해당 페이지의 HTML 문서를 가져오기 위해서 BeatifulSoup, Requests, LXML을 사용했습니다. 

r.headers로 메타데이터를 확인할 수 있습니다.

이제 영화 제목을 가져와야하니, 영화 제목이 html 문서에서 어떤 곳에 표현 되어있는지 확인하겠습니다.

![movie_page.png](movie_page.png)

html은 수많은 태그와 링크들이 나뭇가지처럼 트리구조를 이루고 있습니다.

그리고 BeautifulSoup의 select라는 함수는 수많은 가지들 중 하나의 잎을 찾을 수 있게 해줍니다.

원하는 잎을 찾기 위해선 그 잎이 가지고 있는 고유의 class name, xpath 중 하나를 알아야합니다.

하지만 이 모든게 명확하지 않을 때에는 **태그들이 이루는 구조**를 알고 있으면 원하는 잎을 찾을 수 있습니다.

**영화 제목은 특별한 class name을 가지고 있지 않습니다. 그래서 그 상위에 있는 태그의 class로 수사망을 좁혀보겠습니다.**

In [3]:
title = page.select('div[class=mv_info] a')
print(type(title))
print(len(title))

<class 'list'>
64


**BeautifulSoup.select() 메서드 안에 parameter는 찾고자 하는 정보의 태그 경로를 써줍니다.** 처음부터(head 태그부터) 다 써줄 필요는 없습니다. select 메서드가 혼란을 겪지 않도록 중복되지 않는 class name 부터 경로를 지정해주면 됩니다. (참고로 select 메서드는 html문서를 처음부터 끝까지 scanning하면서 parameter로 받은 경로를 모두 뽑아냅니다.)

div 태그 중 'my_info'라는 class 밑에 있는 a 태그는 총 64개가 나오는군요. 우리는 제목만 필요한데, 제목을 알려주는 경로와 겹치는 정보들이 63개나 더 있습니다.
(아무래도 제목은 특별한 class를 쓰지 않고 그냥 text로 적었다는 것을 예측할 수 있습니다.)

범위를 좁혀볼까요? 잘보면 'my_info'라는 class 밑에 있는 'h_movie'라는 클래스. 그 밑에 있는 a 태그는 두개 밖에 안보이는군요.

그렇다면 어떻게 제목부분만 뽑아낼까. html 문서를 잘 들여다 봅시다.
제목을 적은 a태그는 'h_movie'라는 class 바로 아래에 있습니다.

즉, title에 저장된 2개의 list 중 첫번째 list에 제목이 들어가 있습니다


In [4]:
title = page.select('div[class=mv_info] h3[class=h_movie] a')
print(len(title))

4


출력해보니 같은 경로를 가진 정보가 4개가 있다고 하네요. 그래서 찾아보니 중복된 정보가 하나씩 더 들어가 있네요.

이유는 html문서를 더 파내야 알 수 있겠지만 느낌상 포스터를 클릭했을 때 뜨는 팝업창이나 예매를 누르면 뜨는 팝업창에 같은 정보를 넣기 위함인 것 같습니다.

크롤링은 이렇게 시행착오가 많이 필요합니다.

In [5]:
print(title[0],'\n')

<a href="./basic.nhn?code=156464">보헤미안 랩소디</a> 



원하는 제목부분을 html에서 그대로 떠왔으니 이제 제목부분"만" 남겨야겠죠.

**BeautifulSoup의 메서드 중에서 text를 이용해 태그 안에 있는 text만 남겨보겠습니다.**

In [6]:
print(title[0].text)

보헤미안 랩소디


크롤러은 이렇게 html 구조만 알면 만들기 어렵지 않습니다. 그리고 약간의 인내심이 필요합니다.

잘 만들어져있는 사이트일수록 템플릿을 사용하기 때문에 영화의 모든 제목은 저 태그경로로 모두 뽑아낼 수 있습니다.

## 영화제목 여러개 뽑아오기

하지만 꼭 템플릿이 모두 같다는 보장은 없습니다. 그래서 우리는 크롤링을 할 때 try - except 구문을 자주 사용합니다.
찾고자 하는 태그가 없으면 작업이 끊기지 않고 넘어가죠.

이제 영화제목을 뽑아낼 수 있는 함수를 만들어 여러 페이지에서 실행 시켜보겠습니다.
**원하는 결과가 많을 때는 함수를 여러개로 만들어 정리해 놓는 습관을 들이는게 좋습니다.**

page id를 증가시키면서 다양한 페이지를 볼텐데요. 하지만 page id가 순서대로 모두 존재하지는 않습니다. 그래서 우리는 try - except 구문을 사용할 것이고 없는 페이지는 그냥 넘겨버리죠.

In [0]:
page_url_base = ' https://movie.naver.com/movie/bi/mi/basic.nhn?code=%s'

In [0]:
def get_page(page_url):
    try:
        r = requests.get(page_url)
        return BeautifulSoup(r.text, 'lxml')
    
    ## 페이지가 없을 때
    except Exception as e:
        pprint('오류', e)

In [0]:
def get_title(page):
    try:
        return page.select('div[class=mv_info] h3[class=h_movie] a')[0].text
    except Exception as e:
        pprint('오류', e)
        

In [10]:
for movie_id in range(156454, 156465):
    page_url = page_url_base % movie_id
    page = get_page(page_url)
    title = get_title(page)
    pprint(title)
    

'시각탐정 히구라시 타비토'
'살인편차치70'
'나이프의 행방'
'60분 드라마 2016'
'막부 말기 미식가 무사의 밥'
'시간의 습속'
'리테이크'
'강철의 연금술사'
'나에게 운명의 사랑이란 있을 수 없다고 생각했다'
'무한의 주인'
'보헤미안 랩소디'


이렇게 순식간에, 그리고 보기 편하게 영화들의 제목을 뽑아낼 수 있습니다. 하지만 이렇게 크롤러를 돌리다보면 서버의 트래픽이  굉장히 높아져 서버관리자가 의도적으로 IP를 차단할 수 있습니다. 그렇기 때문에 **적당한 break를 하나의 루프가 돌때마다 적용시켜 크롤링을 하는 것이 바람직합니다.**

In [11]:
import time

for movie_id in range(156454, 156465):
    time.sleep(1)
    
    page_url = page_url_base % movie_id
    page = get_page(page_url)
    title = get_title(page)
    pprint(title)

'시각탐정 히구라시 타비토'
'살인편차치70'
'나이프의 행방'
'60분 드라마 2016'
'막부 말기 미식가 무사의 밥'
'시간의 습속'
'리테이크'
'강철의 연금술사'
'나에게 운명의 사랑이란 있을 수 없다고 생각했다'
'무한의 주인'
'보헤미안 랩소디'
